# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import pprint
import json
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
# Load city data
cities_file_path = "../output_data/cities.csv"

# Read the city data
weather_data_df = pd.read_csv(cities_file_path)

# Display the data table for preview
weather_data_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Windspeed
0,0,Avarua,90,CK,1595287801,74,-21.21,-159.78,78.80,19.46
1,1,Beira,40,MZ,1595287801,94,-19.84,34.84,68.00,4.70
2,2,Carnarvon,0,AU,1595287801,81,-24.87,113.63,51.80,8.05
3,3,Qaanaaq,99,GL,1595287787,71,77.48,-69.36,48.20,1.43
4,4,Longyearbyen,75,SJ,1595287801,87,78.22,15.64,46.40,3.36
...,...,...,...,...,...,...,...,...,...,...
572,572,Luanda,20,AO,1595287875,88,-8.84,13.23,69.80,6.93
573,573,Bemidji,90,US,1595287911,49,47.47,-94.88,75.00,5.82
574,574,Sitka,90,US,1595287911,93,57.05,-135.33,57.20,5.82
575,575,Minot,90,US,1595287898,88,48.23,-101.30,64.40,5.82


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [13]:
# Google developer API key
from config import gkey

# Configure gmaps
gmaps.configure(api_key=g_key)

# Drop NaN values
weather_data_df = weather_data_df.dropna()

# Store 'Lat' and 'Lng' into  locations 
locations = weather_data_df[["Lat", "Lng"]].astype(float)

# Convert Humidity Rate to float and store
humidity_rate = weather_data_df["Humidity"].astype(float)

# Create a Humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity_rate, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
# Create cities dataframe
cities_weather_df = weather_data_df[['City', 'Cloudiness', 'Country', 'Humidity', 'Lat', 'Lng', 'Max Temp', 'Windspeed']]

#  Get the indices of cities that max temp between 70 & 80
cities_weather_df = cities_weather_df[(cities_weather_df['Max Temp']>70) & (cities_weather_df['Max Temp']<80)]

#  Get the indices of cities that have windspeed under 10 mph
cities_weather_df = cities_weather_df[(cities_weather_df['Windspeed']<10)]

#  Get the indices of cities that have windspeed under 10 mph
cities_weather_df = cities_weather_df[(cities_weather_df['Cloudiness'] == 0)]
cities_weather_df.dropna(inplace=True)
cities_weather_df

,City,Cloudiness,Country,Humidity,Lat,Lng,Max Temp,Windspeed
22,Vaini,0,TO,64,-21.20,-175.20,75.20,8.05
107,Dalbandin,0,PK,18,28.89,64.41,79.75,5.17
189,Salamiyah,0,SY,60,35.01,37.05,71.53,7.81
232,Abu Samrah,0,SY,51,35.30,37.18,73.35,8.86
276,Chardara,0,KZ,26,41.25,67.97,78.31,8.19
279,Tripoli,0,LY,77,32.88,13.19,75.70,8.37
307,Tafalla,0,ES,59,42.53,-1.67,73.40,3.36
406,Gönen,0,TR,88,40.10,27.65,71.60,3.36
408,Erdemli,0,TR,71,36.60,34.31,79.77,7.43
459,Marabá,0,BR,69,-5.37,-49.12,77.00,1.30


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
hotel_df = cities_weather_df.copy()
hotel_df['Hotel Name'] = ""

# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    # get hotel from df
    lat = row['Lat']
    lng = row['Lng']
    
    # geocoordinates
    target_coordinates = f"{lat},{lng}"
    target_search = "hotel"
    target_radius = 5000
    target_type = "hotel"
    
    # set up a parameters dictionary
    params = {
    "location": target_coordinates,
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
    }

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        # base url
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

        # run a request using our params dictionary
        response = requests.get(base_url, params=params)

        # convert response to json
        places_data = response.json()

        # Print the json (pretty printed)
        #print(json.dumps(places_data, indent=4, sort_keys=True))

        # Print the name and address of the first restaurant that appears
        #print(places_data["results"][0]["name"])
        #print(places_data["results"][0]["vicinity"])
        hotel_df[index] = places_data["results"][0]["name"]

    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
hotel_df.head()

Missing field/result... skipping.


,City,Cloudiness,Country,Humidity,Lat,Lng,Max Temp,Windspeed,Hotel Name,22,...,189,276,279,307,406,408,459,548,549,555
22,Vaini,0,TO,64,-21.20,-175.20,75.20,8.05,,Paradise First Hotel,...,Resort Safari Souria منتجع سفاري سورية,Жанаргул,Radisson Blu Al Mahary Hotel,Hotel El Juglar,ASEPSUS HOTEL,vm resort,Itacaiúnas Hotel,HOTEL PALACIO PUJADAS,Villa Lattanzi,Free Zone Hotel Nouadhibou
107,Dalbandin,0,PK,18,28.89,64.41,79.75,5.17,,Paradise First Hotel,...,Resort Safari Souria منتجع سفاري سورية,Жанаргул,Radisson Blu Al Mahary Hotel,Hotel El Juglar,ASEPSUS HOTEL,vm resort,Itacaiúnas Hotel,HOTEL PALACIO PUJADAS,Villa Lattanzi,Free Zone Hotel Nouadhibou
189,Salamiyah,0,SY,60,35.01,37.05,71.53,7.81,,Paradise First Hotel,...,Resort Safari Souria منتجع سفاري سورية,Жанаргул,Radisson Blu Al Mahary Hotel,Hotel El Juglar,ASEPSUS HOTEL,vm resort,Itacaiúnas Hotel,HOTEL PALACIO PUJADAS,Villa Lattanzi,Free Zone Hotel Nouadhibou
232,Abu Samrah,0,SY,51,35.30,37.18,73.35,8.86,,Paradise First Hotel,...,Resort Safari Souria منتجع سفاري سورية,Жанаргул,Radisson Blu Al Mahary Hotel,Hotel El Juglar,ASEPSUS HOTEL,vm resort,Itacaiúnas Hotel,HOTEL PALACIO PUJADAS,Villa Lattanzi,Free Zone Hotel Nouadhibou
276,Chardara,0,KZ,26,41.25,67.97,78.31,8.19,,Paradise First Hotel,...,Resort Safari Souria منتجع سفاري سورية,Жанаргул,Radisson Blu Al Mahary Hotel,Hotel El Juglar,ASEPSUS HOTEL,vm resort,Itacaiúnas Hotel,HOTEL PALACIO PUJADAS,Villa Lattanzi,Free Zone Hotel Nouadhibou


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
